In [1]:
import sys 
sys.path.append('../')
from dlim.model import DLIM 
from dlim.dataset import Data_model
from dlim.api import DLIM_API
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import r2_score
from numpy import mean
from numpy.random import choice
import pandas as pd
from tqdm import tqdm  
import matplotlib.pyplot as plt 
import numpy as np 

In [2]:
df_data = pd.read_csv("../data/data_epis_1.csv", sep = ',', header = None)
data = Data_model(data=df_data, n_variables=2)

all_scores = []
all_scores_no = []

pval_l = []
for i in tqdm(range(10)):
    train_id = choice(range(data.data.shape[0]), int(data.data.shape[0]*0.7), replace=False)
    val_id = [i for i in range(data.data.shape[0]) if i not in train_id]
    train_data = data.subset(train_id)
    val_data = data.subset(val_id)
    model = DLIM(n_variables = train_data.nb_val, hid_dim = 32, nb_layer = 1)
    dlim_regressor = DLIM_API(model=model, flag_spectral=True)
    losses = dlim_regressor.fit(train_data, lr = 1e-3, nb_epoch=400, batch_size=32, emb_regularization=1e-2)


    model_no = DLIM(n_variables = train_data.nb_val, hid_dim = 32, nb_layer = 1)
    dlim_regressor_no = DLIM_API(model=model_no, flag_spectral=False)
    losses = dlim_regressor_no.fit(train_data, lr = 1e-3, nb_epoch=400, batch_size=32, emb_regularization=1e-2)


    fit_v, var, _  = dlim_regressor.predict(val_data.data[:,:-1], detach=True) 
    fit_no, var, _  = dlim_regressor_no.predict(val_data.data[:,:-1], detach=True) 

    score, pval = pearsonr(fit_v.flatten(), val_data.data[:, [-1]].flatten())
    score_no, pval = pearsonr(fit_no.flatten(), val_data.data[:, [-1]].flatten())

    all_scores += [score]
    all_scores_no += [score_no]
    pval_l += [pval]
print(mean(all_scores), all(p < 10**-5 for p in pval_l))
print(mean(all_scores_no), all(p < 10**-5 for p in pval_l))

  0%|          | 0/10 [00:00<?, ?it/s]

spectral gap = 0.8882160186767578
spectral gap = 0.6380813121795654


 10%|█         | 1/10 [00:42<06:21, 42.44s/it]

spectral gap = 0.8744031190872192
spectral gap = 0.6912084221839905


 20%|██        | 2/10 [01:25<05:40, 42.54s/it]

...

spectral gap = 0.9083172082901001
spectral gap = 0.6596746444702148


 90%|█████████ | 9/10 [06:12<00:39, 39.70s/it]

spectral gap = 0.8730735778808594
spectral gap = 0.6776626110076904


100%|██████████| 10/10 [06:48<00:00, 40.80s/it]

0.9772620317514237 True
0.9707883365728966 True


In [ ]:
from scipy.stats import ttest 
